In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Upload to Elasticsearch").setMaster("spark://25.15.27.228:7077")
sc = SparkContext.getOrCreate(conf=conf)
print('hello')

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Upload to Elasticsearch').getOrCreate()

In [3]:
URI           = sc._gateway.jvm.java.net.URI
Path          = sc._gateway.jvm.org.apache.hadoop.fs.Path
FileSystem    = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
Configuration = sc._gateway.jvm.org.apache.hadoop.conf.Configuration


In [4]:
fs = FileSystem.get(URI("hdfs://cris:9000"), Configuration())

status = fs.listStatus(Path('/shopee/Product'))
pathProducts = []
for fileStatus in status:
    pathProducts.append(fileStatus.getPath())

In [5]:
pathProducts = []
for fileStatus in status:
    pathProducts.append(str(fileStatus.getPath()))

In [6]:
pathProducts[0]

'hdfs://cris:9000/shopee/Product/_SUCCESS'

In [7]:
pathProducts = pathProducts[1:]

In [89]:
url = pathProducts[0]
parDF2=spark.read.parquet(url)

In [90]:
# data = parDF2.toJSON().collect()

In [91]:
cols = ['historical_sold','raw_discount', 'attributes','shop_location','show_free_shipping', 'name','cmt_count', 'price', 'item_rating','show_shopee_verified_label','liked_count']

In [92]:
tmp = parDF2[cols].toPandas()

In [93]:
def processProduct(data):
    data['price'] = data['price'].astype(float)/100000
    data['rating_star'] = data.item_rating.apply(lambda x: eval(x)['rating_star'])
    data.drop(columns=['attributes','item_rating'],inplace=True)
    return data

In [94]:
data = processProduct(tmp)

In [96]:
data = spark.createDataFrame(data)

In [97]:
data = data.toJSON().collect()

In [98]:
from elasticsearch import Elasticsearch, helpers
import csv

In [2]:
es = Elasticsearch(host = "25.19.134.189", port = 9200,timeout=30,max_retries=10, retry_on_timeout=True)

In [69]:
i = 1
for item in data :

    es.index(index='testv4', doc_type='data', id=i, body=item)

    i = i + 1

<ipython-input-69-10ed49025917>:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='testv4', doc_type='data', id=i, body=item)
